In [1]:
%load_ext autoreload
%autoreload 2
import torch

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt, calculate_metrics_side_effects, create_df_from_metrics, calculate_metrics_list
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np

from jaxtyping import Float, Int
from torch import Tensor
from datasets import load_dataset

import plotly.express as px

from transformer_lens import HookedTransformer
from dataclasses import dataclass
import wandb
import einops
from tqdm import tqdm
import json
import gc
import os


from functools import partial
from unlearning.intervention import anthropic_remove_resid_SAE_features, remove_resid_SAE_features, anthropic_clamp_resid_SAE_features
from unlearning.jump_relu import load_gemma2_2b_sae
from unlearning.metrics import calculate_MCQ_metrics

from transformers import AutoTokenizer, AutoModelForCausalLM

torch.set_grad_enabled(False)

In [3]:
model_name = "google/gemma-2-2b-it"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto").to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)


model-00001-of-00002.safetensors:  49%|####9     | 2.46G/4.99G [00:00<?, ?B/s]

ChunkedEncodingError: ('Connection broken: IncompleteRead(158593496 bytes read, 2365278664 more expected)', IncompleteRead(158593496 bytes read, 2365278664 more expected))

: 

In [135]:
from types import SimpleNamespace

model.tokenizer = tokenizer
model.cfg = SimpleNamespace()
model.cfg.model_name = 'gemma-2-2b-it'

In [115]:
sae = load_gemma2_2b_sae(layer=3)
sae.to('cuda')

Found SAE with l0=59 at path google/gemma-scope-2b-pt-res/layer_3/width_16k/average_l0_59/params.npz


JumpReLUSAE()

In [111]:
def get_data(forget_corpora, retain_corpora, min_len=50, max_len=2000, batch_size=4):
    def get_dataset(name):
        data = []
        if name == "wikitext":
            raw_data = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
            for x in raw_data:
                if len(x['text']) > min_len:
                    data.append(str(x['text']))
        else:
            for line in open(f"../wmdp/data/{name}.jsonl", "r"):
                if "bio-forget-corpus" in name:
                    raw_text = json.loads(line)['text']
                else:
                    raw_text = line
                if len(raw_text) > min_len:
                    data.append(str(raw_text))
        return data

    return get_dataset(forget_corpora), get_dataset(retain_corpora)

forget_dataset, retain_dataset = get_data('bio-forget-corpus', 'wikitext')


In [112]:
print(len(forget_dataset), len(forget_dataset[0]))
print(len(retain_dataset), len(retain_dataset[0]))

24432 16027
1962 859


In [113]:
def tokenize_dataset(model, dataset, seq_len=1024, max_batch=32):
    # just for quick testing on smaller tokens 
    # dataset = dataset[:max_batch]
    full_text = model.tokenizer.eos_token.join(dataset)
    
    # divide into chunks to speed up tokenization
    num_chunks = 20
    chunk_length = (len(full_text) - 1) // num_chunks + 1
    chunks = [full_text[i * chunk_length:(i + 1) * chunk_length] for i in range(num_chunks)]
    tokens = model.tokenizer(chunks, return_tensors="pt", padding=True)["input_ids"].flatten()
    
    # remove pad token
    tokens = tokens[tokens != model.tokenizer.pad_token_id]
    num_tokens = len(tokens)
    num_batches = num_tokens // seq_len
    
    # drop last batch if not full
    tokens = tokens[:num_batches * seq_len]
    tokens = einops.rearrange(
        tokens, "(batch seq) -> batch seq", batch=num_batches, seq=seq_len
    )
    # change first token to bos
    tokens[:, 0] = model.config.bos_token_id
    return tokens.to("cuda")

In [114]:
forget_tokens = tokenize_dataset(model, forget_dataset)
retain_tokens = tokenize_dataset(model, retain_dataset)

print(forget_tokens.shape, retain_tokens.shape)

torch.Size([153108, 1024]) torch.Size([275, 1024])


In [122]:
def gather_residual_activations(model, target_layer, inputs):
    target_act = None
    def gather_target_act_hook(mod, inputs, outputs):
        nonlocal target_act # make sure we can modify the target_act from the outer scope
        target_act = outputs[0]
        return outputs
    handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
    _ = model.forward(inputs)
    handle.remove()
    return target_act

In [116]:
def get_mean_feature_activation(sae, tokens, batch_size=4):
    mean_acts = []
    layer = int(sae.cfg.hook_point.split('.')[1])
    for i in tqdm(range(0, tokens.shape[0], batch_size)):
        with torch.no_grad():
            # _, cache = model.run_with_cache(tokens[i:i + batch_size], names_filter=sae.cfg.hook_name)
            # resid: Float[Tensor, 'batch pos d_model'] = cache[sae.cfg.hook_name]
            resid: Float[Tensor, 'batch pos d_model'] = gather_residual_activations(model, layer, tokens[i:i + batch_size])
            resid = resid.to(torch.float)
            
            act: Float[Tensor, 'batch pos d_sae'] = sae.encode(resid)
            current_mean_act = einops.reduce(act, 'batch pos d_sae -> d_sae', 'mean')
        
        mean_acts.append(current_mean_act)
        
        # Free up memory
        del resid, act
        torch.cuda.empty_cache()
        gc.collect()
        
    mean_acts = torch.stack(mean_acts)
    return mean_acts.to(torch.float16).mean(dim=0).detach().cpu().numpy()

In [117]:
# shuffle forget_tokens 
shuffled_forget_tokens = forget_tokens[torch.randperm(forget_tokens.shape[0])]

mean_feature_activation_forget = get_mean_feature_activation(sae, shuffled_forget_tokens[:2048], batch_size=8)
mean_feature_activation_retain = get_mean_feature_activation(sae, retain_tokens, batch_size=8)

100%|██████████| 35/35 [00:31<00:00,  1.10it/s]


In [118]:

def plot_comparison(forget, retain, good_feature_lst=[]):
    # add color to selected features
    color = np.array(['Normal'] * len(forget))
    color[good_feature_lst] = 'Selected from MCQ'
    
    # main plot
    fig = px.scatter(x=forget, y=retain, labels={'x': 'Forget', 'y': 'Retain'}, hover_data=[np.arange(len(forget))], color=color)
    
    # add a diagonal line
    max_val = min(max(forget), max(retain))
    fig.add_shape(
        type="line", line=dict(dash="dash"),
        x0=0, y0=0, x1=max_val, y1=max_val
    )

    fig.show()
    
    
# plot_comparison(mean_feature_activation_forget, mean_feature_activation_retain)
    

In [119]:
# criteria for selecting features: retain score < 0.01 and then sort by forget score
high_retain_score_features = np.where(mean_feature_activation_retain >= 0.01)[0]
modified_forget_score = mean_feature_activation_forget.copy()
modified_forget_score[high_retain_score_features] = 0
top_features = modified_forget_score.argsort()[::-1]
print(top_features[:20])

n_non_zero_features = np.count_nonzero(modified_forget_score)
top_features_non_zero = top_features[:n_non_zero_features]

[ 8802  1676 10793  8803  1082  8131  9870   570 10387  6798  6396 14059
  1960 10929  8715 12095 11695  9971  1312 16323]


In [120]:
from sae_lens.analysis.neuronpedia_integration import get_neuronpedia_quick_list

get_neuronpedia_quick_list(top_features[:20], layer=3, model='gemma-2-2b', dataset='gemmascope-res-16k')

'https://neuronpedia.org/quick-list/?name=temporary_list&features=%5B%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%228802%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%221676%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%2210793%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%228803%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%221082%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%228131%22%7D%2C%20%7B%22modelId%22%3A%20%22gemma-2-2b%22%2C%20%22layer%22%3A%20%223-gemmascope-res-16k%22%2C%20%22index%22%3A%20%229870%22%7D%2C%20%7B%22modelId%22%3A%20%22ge

In [136]:
from contextlib import contextmanager
from functools import partial

def scale_feature_hook(mod, inputs, outputs, sae, features_to_ablate, multiplier):
    resid = outputs[0].to(torch.float)
    reconstruction = sae(resid)
    feature_activations = sae.encode(resid)
    error = resid - reconstruction
    
    feature_activations[:, :, features_to_ablate] -= multiplier * feature_activations[:, :, features_to_ablate]
    modified_reconstruction = feature_activations @ sae.W_dec + sae.b_dec
    
    resid = error + modified_reconstruction
    return (resid.to(torch.bfloat16), None)

@contextmanager
def scaling_intervention(model, layer, sae, features_to_ablate, multiplier):
    """intervene on resid post at given layer"""
    handle = model.model.layers[layer].register_forward_hook(
        partial(scale_feature_hook, sae=sae, features_to_ablate=features_to_ablate, multiplier=multiplier)
    )
    try:
        yield
    finally:
        handle.remove()



layer = 3
features_to_ablate = list(top_features[:80])
multiplier = 20

for n_features in [20, 40, 60, 80]:
    for multiplier in [0, 10, 20, 40]:
        features_to_ablate = list(top_features[:n_features])
        with scaling_intervention(model, layer, sae, features_to_ablate, multiplier):
            intervened_metrics = calculate_MCQ_metrics(model)
            intervened_history_metrics = calculate_MCQ_metrics(model, dataset_name='high_school_us_history')
            interved_human_aging_metrics = calculate_MCQ_metrics(model, dataset_name='human_aging')
            intervened_college_bio_metrics = calculate_MCQ_metrics(model, dataset_name='college_biology')
            
            print(f"n_features: {n_features}, multiplier: {multiplier}")
            print(f"\t\twmdp-bio: {intervened_metrics['mean_correct']}")
            print(f"\t\thigh_school_us_history: {intervened_history_metrics['mean_correct']}")
            print(f"\t\thuman_aging: {interved_human_aging_metrics['mean_correct']}")
            print(f"\t\tcollege_bio: {intervened_college_bio_metrics['mean_correct']}")
    

100%|██████████| 24/24 [00:02<00:00, 11.01it/s]


n_features: 20, multiplier: 0
		wmdp-bio: 0.6355066895484924
		high_school_us_history: 0.7401961088180542
		human_aging: 0.6322870254516602
		college_bio: 0.7083333134651184


100%|██████████| 24/24 [00:02<00:00, 11.01it/s]


n_features: 20, multiplier: 10
		wmdp-bio: 0.5514532923698425
		high_school_us_history: 0.75
		human_aging: 0.6278027296066284
		college_bio: 0.6527777910232544


100%|██████████| 24/24 [00:02<00:00, 11.00it/s]


...

n_features: 80, multiplier: 10
		wmdp-bio: 0.5003927946090698
		high_school_us_history: 0.7303921580314636
		human_aging: 0.6098654866218567
		college_bio: 0.6458333134651184


100%|██████████| 24/24 [00:02<00:00, 11.01it/s]


n_features: 80, multiplier: 20
		wmdp-bio: 0.38570305705070496
		high_school_us_history: 0.6568627953529358
		human_aging: 0.5515695214271545
		college_bio: 0.3819444477558136


100%|██████████| 24/24 [00:02<00:00, 11.02it/s]

n_features: 80, multiplier: 40
		wmdp-bio: 0.2867242693901062
		high_school_us_history: 0.3970588445663452
		human_aging: 0.47533634305000305
		college_bio: 0.3402777910232544


In [134]:
np.savetxt('./unlearning_output/gemma2-2b-it_layer3_forget_set_features.txt', top_features, fmt='%d')

In [129]:
print(intervened_metrics['mean_correct'])
print(intervened_history_metrics['mean_correct'])

0.5483111143112183
0.7401961088180542


In [124]:
print(intervened_metrics['mean_correct'])
print(intervened_history_metrics['mean_correct'])

0.5200314521789551
0.75


In [126]:
print(intervened_metrics['mean_correct'])
print(intervened_history_metrics['mean_correct'])

0.42262372374534607
0.7450980544090271


In [131]:
print(intervened_metrics['mean_correct'])
print(intervened_history_metrics['mean_correct'])

0.35113903880119324
0.6960784792900085


In [127]:
intervened_metrics

{'mean_correct': 0.42262372374534607,
 'total_correct': 538,
 'is_correct': array([0., 1., 0., ..., 1., 1., 1.], dtype=float32),
 'output_probs': array([[0.02899 , 0.62    , 0.03497 , 0.1776  ],
        [0.05374 , 0.0393  , 0.02104 , 0.373   ],
        [0.01678 , 0.02025 , 0.01392 , 0.917   ],
        ...,
        [0.0923  , 0.0923  , 0.1836  , 0.02333 ],
        [0.0511  , 0.1151  , 0.294   , 0.06555 ],
        [0.00775 , 0.0136  , 0.7427  , 0.008255]], dtype=float16),
 'actual_answers': array([0, 3, 2, ..., 2, 2, 2]),
 'predicted_answers': array([1, 3, 3, ..., 2, 2, 2]),
 'predicted_probs': array([0.62  , 0.373 , 0.917 , ..., 0.1836, 0.294 , 0.7427], dtype=float16),
 'predicted_probs_of_correct_answers': array([0.02899, 0.373  , 0.01392, ..., 0.1836 , 0.294  , 0.7427 ],
       dtype=float16),
 'mean_predicted_prob_of_correct_answers': 0.3037109375,
 'mean_predicted_probs': 0.56787109375,
 'value_counts': {0: 178, 1: 355, 2: 372, 3: 368},
 'sum_abcd': array([0.862 , 0.487 , 0.968 , ..

In [100]:
history_metrics = calculate_MCQ_metrics(model, dataset_name='high_school_us_history')
metrics = calculate_MCQ_metrics(model)

100%|██████████| 34/34 [00:06<00:00,  5.11it/s]
